## Modeling with Julia with Ipopt

Anything with an _a_ reference relates to 'our' team. 
Anything with a _b_ reference relates to 'their' team.

In [1]:
using PyCall
println(PyCall.python)

/Users/lauraroettges/.julia/conda/3/x86_64/bin/python


In [2]:
using PyCall
pyimport_conda("pandas", "pandas")

PyObject <module 'pandas' from '/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/__init__.py'>

In [3]:
using PyCall
pd = pyimport("pandas")

PyObject <module 'pandas' from '/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/__init__.py'>

In [4]:
using DataFrames, PyCall

#Define Data
p_a1 = 0.000518  # (precomputed) probability of score when 5v5
p_b1 = 0.000518  # (precomputed) probability of score when 5v5

p_a2 = 0.00310   # (precomputed) probability that we score during a given second in 6v5 play
p_b2 = 0.00851   # (precomputed) probability that the opponent scores during a given second in 6v5 play

T = 1200 # number of seconds remaining in the game at start of period 3 (20 minutes)

comb_df = pd.read_pickle("../precompute_combinations/precomputed_combinations_df.pkl")
println(comb_df.head())

function choose(n, k)
    #found out through trial and error n must be less than 1200 can not = 1200
    index = n * 1201 + k
    #because of julia need to add 1 to index
    index += 1
    println(index)
    return comb_df.loc[index]["n_choose_k"]
end


#A_0 = [] #TODO: stochastic set -  Our team’s current score (before decision)
#B_0 = [] #TODO: stochastic set -  Their team’s current score (before decision)

scenarios = [:tied, :behind_one, :behind_two]
strategies = [:pull_goalie, :play_normal]

a_0 = Dict(zip(scenarios, [0, 0, 0]))
b_0 = Dict(zip(scenarios, [0, 1, 2]))

#going to start by assuming we are in the behind 1 scenario
A_0 = a_0[:behind_one]
B_0 = b_0[:behind_one]

M = 3599 #be the maximum possible value of d (if their team scored every second of the game and our team never scored)


PyObject            n  k n_choose_k
new_index                 
0          0  0          1
1201       1  0          1
1202       1  1          1
2402       2  0          1
2403       2  1          2


3599

In [53]:
#get unique values of of the indices of comb_df
index_keys = comb_df.index
println(index_keys)
unique_values = comb_df["k"].unique()
println(unique_values)
#test = choose(5, 3)
#(0, 0, 1, 0, 0)
#s, ga1, gb1, ga2, gb2
#choose(s, ga_before)
#choose(s, gb_before)
#choose(T-s, ga2)
#choose(T-s, gb2)
#T = 1200
#(1, 0, 1, 0, 0)



PyObject Index([      0,    1201,    1202,    2402,    2403,    2404,    3603,    3604,
          3605,    3606,
       ...
       1442391, 1442392, 1442393, 1442394, 1442395, 1442396, 1442397, 1442398,
       1442399, 1442400],
      dtype='object', name='new_index', length=721801)
PyObject[PyObject 0, PyObject 1, PyObject 2, PyObject 3, PyObject 4, PyObject 5, PyObject 6, PyObject 7, PyObject 8, PyObject 9, PyObject 10, PyObject 11, PyObject 12, PyObject 13, PyObject 14, PyObject 15, PyObject 16, PyObject 17, PyObject 18, PyObject 19, PyObject 20, PyObject 21, PyObject 22, PyObject 23, PyObject 24, PyObject 25, PyObject 26, PyObject 27, PyObject 28, PyObject 29, PyObject 30, PyObject 31, PyObject 32, PyObject 33, PyObject 34, PyObject 35, PyObject 36, PyObject 37, PyObject 38, PyObject 39, PyObject 40, PyObject 41, PyObject 42, PyObject 43, PyObject 44, PyObject 45, PyObject 46, PyObject 47, PyObject 48, PyObject 49, PyObject 50, PyObject 51, PyObject 52, PyObject 53, PyObject 54, Py

$\mathbb{P}[\mathscr{E}_i] = \mathbb{P}[(G_{a1} == g_{a1}) \& (G_{a2} == g_{a2}) \& (G_{b1} == g_{b1}) \& (G_{b2} = g_{b2})]$ \\
    $ = {s \choose g_{a1}}(p_{a1})^{g_{a1}}(1 - p_{a1})^{s - g_{a1}} \cdot {s \choose g_{b1}}(p_{b1})^{g_{b1}}(1 - p_{b1})^{s - g_{b1}} \cdot {T - s \choose g_{a2}}(p_{a2})^{g_{a2}}(1 - p_{a2})^{T - s - g_{a2}} \cdot {T - s \choose g_{b2}}(p_{b2})^{g_{b2}}(1 - p_{b2})^{T - s - g_{b2}}$

Need to break this down:
${s \choose g_{a1}}(p_{a1})^{g_{a1}}(1 - p_{a1})^{s - g_{a1}} \cdot {s \choose g_{b1}}(p_{b1})^{g_{b1}}(1 - p_{b1})^{s - g_{b1}} \cdot {T - s \choose g_{a2}}(p_{a2})^{g_{a2}}(1 - p_{a2})^{T - s - g_{a2}} \cdot {T - s \choose g_{b2}}(p_{b2})^{g_{b2}}(1 - p_{b2})^{T - s - g_{b2}}$

In [5]:
function probability_of_event(s, T, prob_a1, prob_b1, prob_a2, prob_b2, ga_before, gb_before, ga2, gb2)
    
    #s = number of seconds in epoch 1 (5v5)
    #T = number of seconds in epoch 2 (6v5)
    #prob_a1 = probability of scoring in epoch 1 (5v5)
    #prob_b1 = probability of opponent scoring in epoch 1 (5v5)
    #prob_a2 = probability of scoring in epoch 2 (6v5)
    #prob_b2 = probability of opponent scoring in epoch 2 (6v5)
    #ga_before = number of goals our team has before goalie pull (epoch 0 & 1)
    #gb_before = number of goals their team has before after goalie pull (epoch 0 & 1)
    #ga2 = number of goals our team has after goalie pull (epoch 2)
    #gb2= number of goals their team has after goalie pull (epoch 2)

    #because indicies 
    return choose(s, ga_before) * prob_a1^ga_before * (1 - prob_a1)^(s - ga_before) * choose(s, gb_before) * prob_b1^gb_before * (1 - prob_b1)^(s - gb_before) * choose(T-s, ga2) * prob_a2^ga2 * (1 - prob_a2)^(T - s - ga2) * choose(T-s, gb2) * prob_b2^gb2 * (1 - prob_b2)^(T - s - gb2)
end

probability_of_event (generic function with 1 method)

In [6]:
# Generate all possible states
# Some extra notes:
#important they must all be ints and they must all be >= 0
# 0 =< ga1 <= s #assuming we can't score more than the number of seconds in epoch 1 (5v5) 
# 0 =< gb1 <= s #assuming we can't score more than the number of seconds in epoch 1 (5v5) 

# 0 =< ga2 <= T-s #assuming we can't score more than the number of seconds in epoch 2 (6v5)
# 0 =< gb2 <= T-s #assuming we can't score more than the number of seconds in epoch 2 (6v5) 

#if we want to make s interval a minute for computational tractability could do T = 5 * 60 = 300 (5 minutes)

#will start with just 2 minutes
T = 2 * 60  #2 minutes
states = []
for s in 1:T
    for ga1 in 0:s
        for gb1 in 0:s
            for ga2 in 0:T-s
                for gb2 in 0:T-s
                    push!(states, (s, ga1, gb1, ga2, gb2))
                end
            end
        end
    end
end


In [44]:

# Generate probabilities
probabilities = Dict()
#effectively could precompute w 
win_probs = Dict()
for state in states
    s, ga1, gb1, ga2, gb2 = state
    success = ga1 + ga2 >= gb1 + gb2
    println("state: ", state)
    prob = probability_of_event(s, T, p_a1, p_b1, p_a2, p_b2, ga1, gb1, ga2, gb2)
    println("prob: ", prob)
    probabilities[state] = prob
    if success == true
        win_probs[state] = 1
    else 
        win_probs[state] = 0
    end
end
;

state: (1, 0, 1, 0, 0)
1203
1204


LoadError: PyError ($(Expr(:escape, :(ccall(#= /Users/lauraroettges/.julia/packages/PyCall/1gn3u/src/PyCall.jl:783 =# @pysym(:PyObject_GetItem), PyPtr, (PyPtr, PyPtr), o, PyObject(k)))))) <class 'KeyError'>
KeyError(1203)
  File "/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/core/indexing.py", line 1191, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/core/indexing.py", line 1431, in _getitem_axis
    return self._get_label(key, axis=axis)
  File "/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/core/indexing.py", line 1381, in _get_label
    return self.obj.xs(label, axis=axis)
  File "/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/core/generic.py", line 4301, in xs
    loc = index.get_loc(key)
  File "/Users/lauraroettges/.julia/conda/3/x86_64/lib/python3.10/site-packages/pandas/co

In [ ]:
# I want to save this dictionary to a file so I can load it later
using Serialization

# Save the dictionary to a file
open("probabilities_dict.jls", "w") do io
    serialize(io, probabilities)
end
open("success_state_dict.jls", "w") do io
    serialize(io, win_probs)
end

#may also want to save as a JSON file as back up 
using JSON

# Save the dictionary to a file
open("probabilities_dict_backup.json", "w") do io
    JSON.print(io, probabilities)
end
open("success_state_dict_backup.json", "w") do io
    JSON.print(io, win_probs)
end

In [ ]:
using Serialization
# Load the dictionary from the file
probabilities_loaded = open("probabilities_dict.jls", "r") do io
    deserialize(io)
end

In [ ]:
using JuMP, Ipopt

mod = Model()

#Variables
#assuming we don't choose s = 0 (i.e. pull the goalie immediately - TODO: check w/ Ryan) 
@variable(mod, 1 <= s <= T, Int)      # s: how many seconds to wait before pulling the goalie 
@variable(mod, y, Bin)          # y: indicator variable for final score difference
                                # y = 1 => d is positive (indicating a win or tie), y = 0 => or negative (indicating a loss)
#@variable(mod, d, Int)          # d: final score difference (our team - their team)
@variable(mod, w[1:length(states)] >= 0) # w: probability of each scenario    

#Constraints
# Define constraints based on states (s, ga1, gb1, ga2, gb2)
for (i, state) in enumerate(states)
    s, ga1, gb1, ga2, gb2 = state
    d = (A0 +ga1 + ga2) - (B0 + gb1 + gb2)  # Calculate the score difference
    
    @constraint(mod, w[i] <= probabilities[state])
    @constraint(mod, w[i] >= d)  
    @constraint(mod, w[i] <=  M - (M * y))
    @constraint(mod, w[i] <= d + M * y)
end

# Objective
@objective(model, Max, sum(w[i] for i in 1:length(scenarios)))


# will need the probability distribution for each possible decision

In [ ]:

#set solver
set_optimizer(mod, Ipopt.Optimizer)

# Solve the model
optimize!(mod)

# Print the results
println("Objective value: ", objective_value(mod))
println("pull out the goalie at time s: ", value(s))

In [ ]:
##Code for debugging
## Print the DataFrame's index and columns for debugging
#println("DataFrame Index: ", comb_df.index)
#println("DataFrame Columns: ", comb_df.columns)
#println("Number of Rows: ", comb_df.shape[1])
#
## Access a specific row
#row = comb_df.loc[1202]
#println("this row put in 1202:", row)
#row = comb_df.loc[1203]
#println("next row put in 1203: ", row)
#row = comb_df.loc[2403]
#println("3rd row put in 2403: ",row)
#
## Access a specific element (ensure the index exists and the column name is correct)
#println("this other thing: ",comb_df.loc[1202]["n_choose_k"])
#
#test_mult = comb_df.loc[1202]["n_choose_k"] * .05
#println("test_mult: ", test_mult)